In [2]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import decoupler as dc
import os
#import pyreadr

In [ ]:
R_input = pd.read_csv("LR2TF_test_run\\resultsCrossTalkeR_input_control.csv")

R_input = R_input.where(R_input["MeanLR"] > 0, np.nan) 
R_input.dropna(inplace=True)


R_input.to_csv("R_CRT_no_negative_score.csv")

In [ ]:
import pyreadr
ligands_readin = pyreadr.read_r("ligands_human.rda")
ligands = ligands_readin["ligands_human"]
ligands.to_csv("ligands_human.csv", index=False)

ligands_readin_m = pyreadr.read_r("converted_ligands.rda")
print (ligands_readin_m)
ligands_m = ligands_readin_m["converted_ligands"]
ligands_m.to_csv("converted_ligands.csv", index=False)

rtf_readin = pyreadr.read_r("RTF_DB_2.rda") 
print(rtf_readin)
rtf_db = rtf_readin["RTF_DB_2"]
rtf_db.to_csv("rtf_db.csv", index=False)

rtf_m_readin = pyreadr.read_r("RTF_DB_mouse.rda") 
print (rtf_m_readin)
rtf_db_m = rtf_m_readin["RTF_DB_mouse"]
rtf_db_m.to_csv("rtf_db_mouse.csv", index=False)

In [ ]:
def generate_CrossTalkeR_input(tf_activities, gene_expression, out_path, regulon = None):

  ligands = pd.read_csv("ligands_human.csv")
  R2TF = pd.read_csv("rtf_db.csv")

  sorted_regulon = regulon[["tf", "target"]]
  #sorted_regulon.rename(columns={"target" : "targets"})
  sorted_regulon = sorted_regulon.set_index("tf")

  R2TF = R2TF.set_index("tf")
  output_df = create_empty_CTR_dataframe()
  tf_l = create_empty_CTR_dataframe()
  r_tf = create_empty_CTR_dataframe()

  for row in range(len((tf_activities))):

    #if (tf_activities["z_score"].iloc[row] > 0):
    tf = str(tf_activities["gene"].iloc[row])
    if tf in sorted_regulon.index:
      targets = sorted_regulon.loc[tf, "target"]

    if tf in R2TF.index:
        receptors = R2TF.loc[tf, "receptor"]

    else:
        receptors = []
    tf_ligands = np.intersect1d(targets, ligands)

    if len(tf_ligands) > 0:
      for ligand in tf_ligands:
          expressed = False
          if ligand in gene_expression.index:
            ex_value = gene_expression.loc[ligand, tf_activities.iloc[row, 2]]
            if (ex_value != 0):
              expressed = True
          
          df_list_l = []
            
          if (expressed == True):
            df_list_l = add_entry_to_CTR_dataframe(source = tf_activities.iloc[row, 2],
                                                    target = tf_activities.iloc[row, 2],
                                                    gene_A =tf_activities.iloc[row, 0],
                                                    gene_B = ligand,
                                                    type_gene_A = "Transcription_Factor",
                                                    type_gene_B= "Ligand",
                                                    MeanLR= tf_activities.iloc[row, 3],
                                                    row= row)
              
            tf_l = pd.concat([tf_l, df_list_l], ignore_index=True)
            
    
      if (len(receptors) > 0):
        for receptor in receptors:
          df_list_r = [] 
          df_list_r = add_entry_to_CTR_dataframe(tf_activities.iloc[row, 2],
                                              tf_activities.iloc[row, 2],
                                              receptor,
                                              tf_activities.iloc[row, 0],
                                              "Receptor",
                                              "Transcription Factor",
                                              tf_activities.iloc[row, 3],
                                              row)
              
          print(df_list_r)
            
          r_tf = pd.concat([r_tf, df_list_r], ignore_index=True)

      #tf_l.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_l.csv", index=0)
      #r_tf.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_r.csv", index=0)

  #    r_tf["gene_A"] <- gsub("_", "+", r_tf$gene_A, fixed = TRUE)
   #   r_tf["gene_B"] <- gsub("_", "+", r_tf$gene_B, fixed = TRUE)
   #   tf_l["gene_A"] <- gsub("_", "+", tf_l$gene_A, fixed = TRUE)
   #   tf_l["gene_B"] <- gsub("_", "+", tf_l$gene_B, fixed = TRUE)

  output_df = pd.concat([output_df, r_tf], ignore_index=True)
  output_df = pd.concat([output_df, tf_l], ignore_index=True)
  #output_df.to_csv("tf_l_r.csv", index=0)


  return output_df

In [ ]:
def map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers):
    z_score = []
    for gene in anndataobject_markers["gene"]:
        if gene in filtered_summarized_tf_scores_df.index:
            z_score.append(filtered_summarized_tf_scores_df.loc[gene, anndataobject_markers["cluster"]])
    return z_score


In [ ]:
#comparison python wilcoxon and r wilcoxon SPECIFIC MARKERS

py_result_wilcx = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_wilcoxon_test.csv")
r_result_wilcx = pd.read_csv("LR2TF_test_run\\results\\TF_results\\control\\all_specificmarker__control.csv")

r_result_wilcx = r_result_wilcx[r_result_wilcx["p_val_adj"] < 0.05]

py_result_wilcx = py_result_wilcx.rename(columns={"names": "gene", "group": "cluster"})

df_output_py = pd.DataFrame()

for i in range(len(py_result_wilcx)):
    a = py_result_wilcx["gene"].iloc[i]
    for j in range(len(r_result_wilcx)):
        b = r_result_wilcx["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result_wilcx.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result_wilcx)):
    a = r_result_wilcx["gene"].iloc[i]
    for j in range(len(py_result_wilcx)):
        b = py_result_wilcx["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result_wilcx.iloc[i]

df_output_r = df_output_r.T

df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
#df_output = py_result_wilcx.rename(columns={"gene": "gene_py", "Unnamed: 0": "gene_R"})

df_output.to_csv("tf_comparison_r_py/py_wilcoxon_r_same_genes_control_filtered.csv")

#comparison python t test and r wilcoxon

py_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_t_test_overestim_test.csv")
r_result = pd.read_csv("LR2TF_test_run\\results\\TF_results\\control\\all_specificmarker__control.csv")

py_result = py_result[py_result["pvals_adj"] < 0.05]
r_result = r_result[r_result["p_val_adj"] < 0.05]
py_result = py_result.rename(columns={"names": "gene", "group": "cluster"})


df_output_py = pd.DataFrame()

for i in range(len(py_result)):
    a = py_result["gene"].iloc[i]
    for j in range(len(r_result)):
        b = r_result["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result)):
    a = r_result["gene"].iloc[i]
    for j in range(len(py_result)):
        b = py_result["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result.iloc[i]

df_output_r = df_output_r.T

#df_output = pd.concat([df_output_py, df_output_r], keys= ["gene"])
df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
#df_output = py_result_wilcx.rename(columns={"gene": "gene_py", "Unnamed: 0": "gene_R"})

df_output.to_csv("tf_comparison_r_py/py_t_test_r_same_genes_control_filtered.csv")

# py wilcox vs py t test
py_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_t_test_overestim_test.csv")
r_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_wilcoxon_test.csv")


py_result = py_result[py_result["pvals_adj"] < 0.05]
r_result = r_result[r_result["p_val_adj"] < 0.05]
py_result = py_result.rename(columns={"names": "gene", "group": "cluster"})


df_output_py = pd.DataFrame()

for i in range(len(py_result)):
    a = py_result["gene"].iloc[i]
    for j in range(len(r_result)):
        b = r_result["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result)):
    a = r_result["gene"].iloc[i]
    for j in range(len(py_result)):
        b = py_result["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result.iloc[i]

df_output_r = df_output_r.T

df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
df_output.to_csv("tf_comparison_r_py/py_t_test_py_wilcox_same_genes_control_filtered.csv")

In [ ]:
#comparison python wilcoxon and r wilcoxon CROSSTALKER INPUT

py_result_wilcx = pd.read_csv("script_test\\CrossTalkeR_input_control.csv")
r_result_wilcx = pd.read_csv("LR2TF_test_run\\resultsCrossTalkeR_input_control.csv")


df_output_py = pd.DataFrame()

for i in range(len(py_result_wilcx)):
    a = py_result_wilcx["gene_A"].iloc[i]
    a2 = py_result_wilcx["gene_B"].iloc[i]
    a3 = py_result_wilcx["source"].iloc[i]
    a4 =  py_result_wilcx["target"].iloc[i]
    for j in range(len(r_result_wilcx)):
        b = r_result_wilcx["gene_A"].iloc[j]
        b2 = r_result_wilcx["gene_B"].iloc[j]
        b3 =  r_result_wilcx["source"].iloc[j]
        b4 =  r_result_wilcx["target"].iloc[j]
        if ((a == b) & (a2 == b2) & (a3 == b3) & (a4 == b4)):
            df_output_py[i] = py_result_wilcx.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result_wilcx)):
    a = r_result_wilcx["gene_A"].iloc[i]
    a2 = r_result_wilcx["gene_B"].iloc[i]
    a3 = r_result_wilcx["source"].iloc[i]
    a4 =  r_result_wilcx["target"].iloc[i]
    for j in range(len(py_result_wilcx)):
        b = py_result_wilcx["gene_A"].iloc[j]
        b2 = py_result_wilcx["gene_B"].iloc[j]
        b3 =  py_result_wilcx["source"].iloc[j]
        b4 =  py_result_wilcx["target"].iloc[j]
        if ((a == b) & (a2 == b2) & (a3 == b3) & (a4 == b4)):
            df_output_r[i] = r_result_wilcx.iloc[i]

df_output_r = df_output_r.T

df_output = pd.merge(df_output_py, df_output_r, on="gene_A", suffixes=("_py", "_r"))
#df_output = py_result_wilcx.rename(columns={"gene": "gene_py", "Unnamed: 0": "gene_R"})

df_output.to_csv("tf_comparison_r_py/py_r_crosstalker_input.csv")



In [2]:
import pandas as pd

# Read input CSVs
py_result_wilcx = pd.read_csv("script_test/CrossTalkeR_input_PMF_MF2.csv")
r_result_wilcx = pd.read_csv("LR2TF_test_run/resultsCrossTalkeR_input_PMF_MF2.csv")

# Merge DataFrames on the specified columns
common_columns = ["source", "target", "gene_A", "gene_B"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, on=common_columns, suffixes=("_py", "_r"))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/py_r_crosstalker_input_PMF.csv", index=False)

In [4]:
import pandas as pd

# Read input CSVs
py_result_wilcx = pd.read_csv("script_test/CrossTalkeR_input_control.csv")
r_result_wilcx = pd.read_csv("R_CRT_no_negative_score.csv")

# Merge DataFrames on the specified columns
common_columns = ["source", "target", "gene_A", "gene_B"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, on=common_columns, suffixes=("_py", "_r"))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/py_r_crosstalker_input_control_no_neg_score.csv", index=False)

In [2]:
import pandas as pd

# Read input CSVs
r_result_wilcx = pd.read_csv("r_avg_expr_ctrl.csv")
py_result_wilcx = pd.read_csv("script_test/control_average_gene_expression_by_cluster_exp.csv")

# Merge DataFrames on the specified columns
common_columns = ["Neural", "MSC", "Fibroblast", "Megakaryocyte", "Myeloid"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, left_index=True,right_index=True, suffixes=("_py", "_r"))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/avg_gene_expr_ctrl.csv", index=False)

In [7]:
import pandas as pd

# Read input CSVs
py_result_wilcx = pd.read_csv("script_test/CrossTalkeR_input_PMF_MF2.csv")
r_result_wilcx = pd.read_csv("LR2TF_test_run/resultsCrossTalkeR_input_PMF_MF2.csv")

# Merge DataFrames on the specified columns
common_columns = ["gene_A", "gene_B"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, on=common_columns, suffixes=("_py", "_r"))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/py_r_crosstalker_input_genes_PMF.csv", index=False)

In [ ]:

    
    #  df = add_entry_to_CTR_dataframe(tf_activities.iloc[row, 2],
    #                                         tf_activities.iloc[row, 2],
    #                                         tf_activities.iloc[row, 0],
    #                                         ligand,
    #                                         'Transcription Factor',
    #                                         'Ligand',
     #                                        tf_activities.iloc[row, 3])
      
 #     

In [ ]:
gene_ids = sub_object.var.index.values
    #cluster line even necessary if not returned?
    #clusters = anndataobject.obs[celltype].cat.categories
obs = sub_object[:,gene_ids].X.toarray()
obs = np.expm1(obs)
sub_object = pd.DataFrame(obs,columns=gene_ids,index= sub_object.obs["new_annotation"])
sub_object = sub_object.groupby(level=0, observed=False).mean()
#sub_object.T.to_csv(outpath + name + "average_gene_expression_by_cluster_exp.csv")

sub_object.T


In [ ]:
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")
      #  anndataobject.rename_categories("protocol", ["PMFMF2", "control"])

tf_activities = ad.read_csv("decoupler_results.csv")
tf_activities.obsm = anndataobject.obsm
tf_activities.uns = anndataobject.uns
tf_activities.obs = anndataobject.obs

tf_activities_sub = tf_activities[anndataobject.obs.protocol == "PMF,MF2"]
#print(tf_activities_sub.var.index.values)
#print(tf_activities_sub.var_names)

tf_act_2 = tf_activities_sub
tf_act_2.obs_names = tf_act_2.obs["new_annotation"]
tf_df = tf_act_2.to_df()
tf_df.groupby("new_annotation").mean().T

In [ ]:
tf_activities = ad.read_csv("decoupler_results.csv")
tf_activities.obsm = anndataobject.obsm
tf_activities.uns = anndataobject.uns
tf_activities.obs = anndataobject.obs
tf_activities = tf_activities[tf_activities.obs.protocol == "control"]
tf_activities.obs

In [ ]:
#tag mapping t over
    tag_mapping_t_over= anndataobject_markers_t_over[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]

    tag_mapping_t_over = tag_mapping_t_over[(tag_mapping_t_over["pvals_adj"] < float(pval)) & 
                              ((tag_mapping_t_over["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_t_over["logfoldchanges"] < -float(logfc)))]

     # Pivot the DataFrame to create a wide format similar to `dcast` in R
    tag_mapping_t_over = tag_mapping_t_over.pivot(index="gene", columns="cluster", values="tag")
    tag_mapping_t_over.fillna("ns", inplace=True)

In [ ]:
# Summarize viper_scores_clusters by TF and cell type: compute mean and std (standard deviation)
    #summarized_viper_scores = (viper_scores_clusters
       # .groupby(['tf', 'cell_type'])  # Group by transcription factor and cell type
     #   .agg(avg=('activity', 'mean'), std=('activity', 'std'))  # Calculate mean and standard deviation
      #  .reset_index())

# Spread the summarized data to wide format: remove 'std', and spread 'tf' on columns, 'avg' as values
   # summarized_viper_scores_df = (summarized_viper_scores
   #     .drop(columns=['std'])  # Remove the std column
   #     .pivot(index='cell_type', columns='tf', values='avg')  # Spread the TFs as columns, average activity as values
    #    .reset_index(drop=True))  # Reset the index to make it a regular DataFrame

# Transpose the dataframe (same as R's t())
   # summarized_viper_scores_df = summarized_viper_scores_df.T

# Write the summarized TF scores to CSV

In [ ]:
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")


tf_activities = ad.read_csv("decoupler_results.csv")
anndataobject.obsm['ulm_estimate'] = tf_activities
print(anndataobject.obsm["ulm_estimate"].X)
#for name in anndataobject.obs.protocol.unique():
##    subobject = anndataobject[anndataobject.obs.protocol == name]
#print(type(subobject)) 
#print(type(anndataobject))  
#print(subobject.X)



In [ ]:
res[i] = {}
                    for pw in pws:
                        group1 = a_sub[:, pw].X[g == vs1]
                        group2 = a_sub[:, pw].X[g == vs2]

                    
                        if len(group1) > 0 and len(group2) > 0:
                        # Wilcoxon rank-sum test
                            stat, pval = ranksums(group1, group2)

                        # Calculate effect size
                            r = abs(stat) / np.sqrt(len(group1) + len(group2))  # approximate r calculation
                        
                            res[i][pw] = {"pval": pval, "r": r}

In [ ]:
def map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers):
    genes = []
    clusters = []
    z_scores = []  

    anndataobject_markers = anndataobject_markers.set_index("gene")
    #print(filtered_summarized_tf_scores_df)
    
    for i in range(len(anndataobject_markers.index)):
        a = anndataobject_markers.index[i]
        for j in range(len(filtered_summarized_tf_scores_df.index)):
            b = filtered_summarized_tf_scores_df.index[j]
            if a == b:
                c = anndataobject_markers.columns.get_loc("cluster")
                cluster = anndataobject_markers.iloc[i, c]
                gene_rows = filtered_summarized_tf_scores_df.iloc[j]
                if isinstance(gene_rows, pd.Series):
                    gene_rows = gene_rows.to_frame().T

                for _, gene_row in gene_rows.iterrows():
                     score = gene_row[cluster]
                     genes.append(a)
                     clusters.append(cluster)
                     z_scores.append(score)

    z_score_df = pd.DataFrame({
        'gene': genes,
        'cluster': clusters,
        'z_score': z_scores
    })

    return z_score_df

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
from scipy.stats import ranksums

vs_df_list = []

for vs in comparison_list:
    vs1, vs2 = vs[0], vs[1]

    print(f"vs: {vs1} {vs2} {pd.Timestamp.now()}\n")
    
    # Assuming `seuratobject` is an AnnData object
    condition_annotation = "tf_condition"  # replace with actual column name
    tf_annotation = "tf_annotation"        # replace with actual column name

    pws = seuratobject.var_names  # Assuming the variable features correspond to `tf_activities`

    # Result dictionary
    res = {}
    for i in seuratobject.obs[tf_annotation].unique():
        # Subsetting the AnnData object
        subset_mask = (seuratobject.obs[tf_annotation] == i) & \
                      (seuratobject.obs[condition_annotation].isin([vs1, vs2]))
        a_sub = seuratobject[subset_mask]

        if len(a_sub.obs[condition_annotation].unique()) == 2:
            # Creating a condition table
            condition_table = a_sub.obs[[condition_annotation]].copy()
            condition_table.columns = ["condition"]
            metadata_counts = condition_table.groupby("condition").size().reset_index(name="total_count")
            
            if all(metadata_counts["total_count"] > 10):
                g = a_sub.obs[condition_annotation].astype("category")
                g = g.cat.set_categories([vs1, vs2])

                # Perform statistical analysis
                res[i] = {}
                for pw in pws:
                    group1 = a_sub[:, pw].X[g == vs1]
                    group2 = a_sub[:, pw].X[g == vs2]

                    if len(group1) > 0 and len(group2) > 0:
                        # Wilcoxon rank-sum test
                        stat, pval = ranksums(group1, group2)

                        # Calculate effect size
                        r = abs(stat) / np.sqrt(len(group1) + len(group2))  # approximate r calculation
                        
                        res[i][pw] = {"pval": pval, "r": r}

                # Convert results to a DataFrame
                res[i] = pd.DataFrame.from_dict(res[i], orient="index")
                res[i].index.name = "pathway"

        from pathlib import Path
import pandas as pd
import numpy as np

vs_df_list = {}

for vs in comparison_list:
    vs1, vs2 = vs[0], vs[1]
    print(f"Processing {vs1} vs {vs2}...")

    for cl in res.keys():
        res[cl]["tf"] = res[cl].index
        res[cl]["CellType"] = cl
        res[cl].columns = [
            "Top", "p.value", "FDR", "summary.logFC", 
            "logFC", "r", "tf", "CellType"
        ]

    # Combine all dataframes in `res` into a single dataframe
    res_df = pd.concat(res.values(), axis=0, ignore_index=True)
    res_df = res_df.dropna()

    # Assign significance tags based on FDR values
    def assign_significance_tag(fdr):
        if fdr < 0.001:
            return "***"
        elif fdr < 0.01:
            return "**"
        elif fdr < 0.05:
            return "*"
        else:
            return "ns"

    res_df["tag"] = res_df["FDR"].apply(assign_significance_tag)

    # Uncomment the following lines to filter for significant results
    # significant_res = res_df[res_df["tag"] == "***"]
    # significant_genes = significant_res["tf"].unique()
    # end_res = res_df[res_df["tf"].isin(significant_genes)]
    end_res = res_df

    # Save the dataframe to CSV
    out_filename = f"{vs1}_vs_{vs2}.csv"
    out_filepath = Path(out_path) / f"all_tfs_{out_filename}"
    end_res.to_csv(out_filepath, index=False)

    # Save to vs_df_list
    comparison_key = f"{vs1} vs {vs2}"
    vs_df_list[comparison_key] = end_res

# Save the entire vs_df_list as a pickle file
comparison_dfs_filepath = Path(out_path) / "comparison_dfs.pkl"
pd.to_pickle(vs_df_list, comparison_dfs_filepath)

print("Processing complete.")



In [ ]:
def get_significant_tfs_single(tf_activities_sub, celltype, condition, out_path, pval, logfc, name, plot):

    renamed_condition = condition.replace(",", "_")
    name = name.replace(",", "_")

    single_result_path = out_path + renamed_condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)

    tf_activities_scaled = sc.pp.scale(tf_activities_sub, copy= True)

    #tf_activities_sub = sc.pp.normalize_total(tf_activities_scaled, copy=True)
    #tf_activities_sub = sc.pp.log1p(tf_activities_sub, copy=True) 
    # "warning: seems to be already log transformed"

    number_of_clusters = len(tf_activities_sub.obs["new_annotation"].cat.categories) 


    sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")
    

    sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers", corr_method= "bonferroni" )
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")


    result1 = tf_activities_sub.uns['wilcoxon_markers']
    groups = result1['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result1[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    result2 = tf_activities_sub.uns['t_test_overestim_var_markers']
    groups = result2['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result2[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="wilcoxon_markers"
    
    
    anndataobject_markers_wilcoxon.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)
    
    #documentation says this won't update the main dataframe but will only overwrite a copy in future pandas versions (after v3)
    anndataobject_markers_wilcoxon["tag"] = None
    anndataobject_markers_wilcoxon["log_fc_tag"] = None
    
    for i in range(len(anndataobject_markers_wilcoxon["pvals_adj"])):
         anndataobject_markers_wilcoxon["tag"].iloc[i,] = eval_pval(anndataobject_markers_wilcoxon["pvals_adj"].iloc[i,])
         anndataobject_markers_wilcoxon["log_fc_tag"].iloc[i,] = eval_log_fc_tag(anndataobject_markers_wilcoxon["logfoldchanges"].iloc[i,])
     

    anndataobject_markers_wilcoxon.to_csv(single_result_path + "/" + name + "_specific_markers_wilcoxon_test.csv",index=0)

    anndataobject_markers_t_over = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
 
    anndataobject_markers_t_over.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)

    anndataobject_markers_t_over["tag"] = None
    anndataobject_markers_t_over["log_fc_tag"] = None
    for i in range(len(anndataobject_markers_t_over["pvals_adj"])):
         anndataobject_markers_t_over["tag"].iloc[i,] = eval_pval(anndataobject_markers_t_over["pvals_adj"].iloc[i,])
         anndataobject_markers_t_over["log_fc_tag"].iloc[i,] = eval_pval(anndataobject_markers_t_over["logfoldchanges"].iloc[i,])

    anndataobject_markers_t_over.to_csv(single_result_path + "/" + name + "_specific_markers_t_test_overestim_test.csv",index=0)

   #tag mapping wilcoxon
    tag_mapping_wilcox = anndataobject_markers_wilcoxon[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["pvals_adj"] < float(pval)) & 
                              ((tag_mapping_wilcox["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_wilcox["logfoldchanges"] < -float(logfc)))]

    tag_mapping_wilcox = tag_mapping_wilcox.pivot(index="gene", columns="cluster", values="tag")
    clusters = anndataobject_markers_wilcoxon["cluster"].unique()

    for cluster in clusters:
        if cluster not in tag_mapping_wilcox.columns:
            tag_mapping_wilcox[cluster] = np.nan

    tag_mapping_wilcox.fillna("ns", inplace=True)

    tf_activities_scaled.obs_names = tf_activities_scaled.obs[celltype]
    tf_scores_df = tf_activities_scaled.to_df()
    tf_scores_df.columns.name = None
    unfiltered_tf_scores = create_unfiltered_tf_scores(tf_scores_df, condition, celltype, single_result_path)
   
    #Filter to only include tfs that match the tag_mapping/are markers
    col_num = tf_scores_df.columns.isin(tag_mapping_wilcox.index)  
    filtered_tf_scores_df = tf_scores_df.loc[:, col_num] 
    
    filtered_summarized_tf_scores_df = filtered_tf_scores_df.groupby(celltype, observed = False).mean().T
    filtered_summarized_tf_scores_df.index.name = "gene"
    filtered_summarized_tf_scores_df.to_csv(f'{single_result_path}/tf_scores_{condition}.csv')
    tf_scores_variable_table = save_variable_tf_score(filtered_summarized_tf_scores_df, condition, single_result_path, plot)

    if plot:
        plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping_wilcox, condition, single_result_path)
    
    #filtered_summarized_tf_scores_df.index = re.sub((".,"), "_", filtered_summarized_tf_scores_df.index)
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon.merge(map_z_value(filtered_summarized_tf_scores_df, anndataobject_markers_wilcoxon),  on=['gene', 'cluster'], how='inner')
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon_merged[anndataobject_markers_wilcoxon_merged.tag != "ns"]
    anndataobject_markers_wilcoxon_merged.dropna(inplace=True)
    res_wilcoxon = anndataobject_markers_wilcoxon_merged[["gene","tag", "cluster", "z_score"]]

    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over.merge(map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers_t_over),  on=['gene', 'cluster'], how='inner')
    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over_merged[anndataobject_markers_t_over_merged.tag != "ns"]
    #anndataobject_markers_t_over_merged.dropna(inplace=True)
    #res_t_test = anndataobject_markers_t_over[["gene","tag", "cluster", "z_score"]]

    res_wilcoxon.to_csv(single_result_path + "/" + name + "_significant_markers_wilcoxon.csv", index=0)
    #res_t_test.to_csv(single_result_path + "/" + name + "_significant_markers_t_test.csv", index=0)

     #//TODO: 
    #delete one variant and put test type as a variable

    res = {}
    res["cluster"] = res_wilcoxon
    return res